In [1]:
import pyrealsense2 as rs
import numpy as np
import cv2
import os
import time
import glob
import yaml

context = rs.context()
devices = context.query_devices()
for dev in devices:
    print(dev.get_info(rs.camera_info.name))

Intel RealSense D405


# 0. 标定
- 标定板
- 相机： Intel RealSense D405 

In [2]:
# 创建管道并配置
pipeline = rs.pipeline()
config = rs.config()

config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)

# 启用红外流并配置Y16格式
# config.enable_stream(rs.stream.infrared, 1, 848, 480, rs.format.y16, 15)
 
# 启动管道
pipe_profile = pipeline.start(config)
# 获取出厂内参矩阵
profile = pipe_profile.get_stream(rs.stream.color)
intrinsics = profile.as_video_stream_profile().get_intrinsics()
 
print(f"焦距: fx={intrinsics.fx}, fy={intrinsics.fy}")
print(f"主点坐标: ppx={intrinsics.ppx}, ppy={intrinsics.ppy}")
print(f"畸变系数: {intrinsics.coeffs}")

焦距: fx=645.2243041992188, fy=644.3118896484375
主点坐标: ppx=633.8898315429688, ppy=360.0555419921875
畸变系数: [-0.05372624471783638, 0.05638597533106804, 0.000343952386174351, 0.0005959899281151593, -0.01772184669971466]


## 0.1 拍照

In [3]:
import os
import time
import re
import cv2
import numpy as np
import pyrealsense2 as rs

# ======================
# 配置参数
# ======================
SAVE_DIR = "Apriltag_images"
IMG_PREFIX = "img"
MAX_IMAGES = 30

WIDTH = 1280
HEIGHT = 720
FPS = 30

# ======================
# 创建保存目录
# ======================
os.makedirs(SAVE_DIR, exist_ok=True)

# ======================
# 读取已有图片数量
# ======================
def get_existing_image_count(save_dir, prefix):
    pattern = re.compile(rf"{prefix}_(\d+)\.png")
    indices = []

    for fname in os.listdir(save_dir):
        match = pattern.match(fname)
        if match:
            indices.append(int(match.group(1)))

    if len(indices) == 0:
        return 0
    else:
        return max(indices) + 1


img_count = get_existing_image_count(SAVE_DIR, IMG_PREFIX)
print(f"[INFO] Found {img_count} existing images. Continue from here.")

# ======================
# RealSense 初始化
# ======================
pipeline = rs.pipeline()
config = rs.config()

config.enable_stream(
    rs.stream.color,
    WIDTH,
    HEIGHT,
    rs.format.bgr8,
    FPS
)

print("[INFO] Starting RealSense pipeline...")
pipeline.start(config)

# 等待自动曝光稳定
for _ in range(30):
    pipeline.wait_for_frames()

print("[INFO] Press 'Enter' to save image, 'Esc' to quit")

try:
    while True:
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        if not color_frame:
            continue

        color_image = np.asanyarray(color_frame.get_data())

        display = color_image.copy()
        cv2.putText(
            display,
            f"Saved: {img_count}/{MAX_IMAGES}",
            (20, 40),
            cv2.FONT_HERSHEY_SIMPLEX,
            1.0,
            (0, 255, 0),
            2
        )
        cv2.putText(
            display,
            "Press 'Enter' to save | 'Esc' to quit",
            (20, HEIGHT - 20),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.8,
            (0, 255, 0),
            2
        )

        cv2.imshow("D405 Capture", display)
        key = cv2.waitKey(1) & 0xFF

        if key == 13:  # Enter
            filename = f"{IMG_PREFIX}_{img_count:03d}.png"
            path = os.path.join(SAVE_DIR, filename)
            cv2.imwrite(path, color_image)
            print(f"[INFO] Saved {path}")
            img_count += 1
            time.sleep(0.3)

        elif key == 27:  # ESC
            break

        if img_count >= MAX_IMAGES:
            print("[INFO] Reached max images.")
            break

finally:
    pipeline.stop()
    cv2.destroyAllWindows()
    print("[INFO] Pipeline stopped.")


[INFO] Found 5 existing images. Continue from here.
[INFO] Starting RealSense pipeline...
[INFO] Press 'Enter' to save image, 'Esc' to quit
[INFO] Saved Apriltag_images\img_005.png
[INFO] Saved Apriltag_images\img_006.png
[INFO] Saved Apriltag_images\img_007.png
[INFO] Pipeline stopped.


## 0.2 计算内参

In [11]:
IMAGE_PATH = "calib_images/*.png"

ROWS = 6   # 棋盘格 内角点 行数（垂直）
COLS = 9   # 棋盘格 内角点 列数（水平）

SQUARE_SIZE =20.0  # mm

OUTPUT_YAML = "d405_rgb_intrinsics.yaml"

# ============================
# 准备世界坐标（Z=0 平面）
# ============================
objp = np.zeros((ROWS * COLS, 3), np.float32)
print(objp.shape)
# print(objp)
objp[:, :2] = np.mgrid[0:COLS, 0:ROWS].T.reshape(-1, 2)
print(objp.shape)
# print(objp)
objp *= SQUARE_SIZE  # 单位：mm
# print(objp)

(54, 3)
(54, 3)


In [12]:
objpoints = []  # 3D 点
imgpoints = []  # 2D 点

# ============================
# 读取图片
# ============================
images = glob.glob(IMAGE_PATH)
assert len(images) > 0, "❌ 没找到标定图片"

print(f"[INFO] Found {len(images)} calibration images")

# ============================
# 棋盘格检测
# ============================
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ret, corners = cv2.findChessboardCorners(
        gray,
        (COLS, ROWS),
        cv2.CALIB_CB_ADAPTIVE_THRESH +
        cv2.CALIB_CB_FAST_CHECK +
        cv2.CALIB_CB_NORMALIZE_IMAGE
    )

    if ret:
        objpoints.append(objp)

        # 亚像素精确化
        corners_sub = cv2.cornerSubPix(
            gray,
            corners,
            winSize=(11, 11),
            zeroZone=(-1, -1),
            criteria=(
                cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER,
                30,
                0.001
            )
        )

        imgpoints.append(corners_sub)

        # 可视化（可按 ESC 跳过）
        cv2.drawChessboardCorners(img, (COLS, ROWS), corners_sub, ret)
        cv2.imshow("Corners", img)
        while True:
            key = cv2.waitKey(0) & 0xFF
            if key == 32:      # 空格键
                break
            elif key == 27:    # ESC 键
                cv2.destroyAllWindows()
                exit(0)
    else:
        print(f"[WARN] Chessboard NOT found in {fname}")

cv2.destroyAllWindows()

# ============================
# 相机标定
# ============================
print("[INFO] Calibrating camera...")

ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(
    objpoints,
    imgpoints,
    gray.shape[::-1],
    None,
    None
)

# ============================
# 计算重投影误差
# ============================
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(
        objpoints[i],
        rvecs[i],
        tvecs[i],
        camera_matrix,
        dist_coeffs
    )
    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2) / len(imgpoints2)
    mean_error += error

mean_error /= len(objpoints)

print("====== Calibration Result ======")
print("Camera Matrix:\n", camera_matrix)
print("Distortion Coefficients:\n", dist_coeffs.ravel())
print(f"Mean reprojection error: {mean_error:.6f} pixels")

# ============================
# 保存到 YAML
# ============================
data = {
    "camera_matrix": camera_matrix.tolist(),
    "dist_coeffs": dist_coeffs.tolist(),
    "reprojection_error": float(mean_error),
    "image_width": gray.shape[1],
    "image_height": gray.shape[0],
    "square_size_mm": SQUARE_SIZE,
    "board_rows": ROWS,
    "board_cols": COLS
}

with open(OUTPUT_YAML, "w") as f:
    yaml.dump(data, f)

print(f"[INFO] Calibration saved to {OUTPUT_YAML}")


[INFO] Found 28 calibration images
[INFO] Calibrating camera...
====== Calibration Result ======
Camera Matrix:
 [[591.36695928   0.         643.85593567]
 [  0.         590.12810273 370.38761824]
 [  0.           0.           1.        ]]
Distortion Coefficients:
 [-0.05380712  0.04227481  0.00049924  0.00054192 -0.01783079]
Mean reprojection error: 0.025221 pixels
[INFO] Calibration saved to d405_rgb_intrinsics.yaml


# 1. AprilTag

In [ ]:
import cv2
import numpy as np
import glob
from pyapriltags import Detector

# =========================
# 参数
# =========================
TAG_SIZE = 0.005  # 5 mm (meters)

fx, fy, cx, cy = (
    591.36695928,
    590.12810273,
    643.85593567,
    370.38761824
)

K = np.array([
    [fx, 0, cx],
    [0, fy, cy],
    [0,  0,  1]
])

# =========================
# 初始化 AprilTag Detector
# =========================
detector = Detector(
    families='tag36h11',  # 编码族
    nthreads=2, # 线程数
    quad_decimate=1.0, # 图像不降采样
    quad_sigma=0.0, # 高斯模糊的 σ
    refine_edges=1, # 是否优化边缘
    decode_sharpening=0.25 # 对比度增强
)

# =========================
# Tag 的 3D 角点（tag 坐标系）
# 顺序需和 det.corners 一致
# =========================
half = TAG_SIZE / 2
tag_points_3d = np.array([
    [-half,  half, 0],
    [ half,  half, 0],
    [ half, -half, 0],
    [-half, -half, 0]
], dtype=np.float32)

# =========================
# 批量读取图片
# =========================
image_files = sorted(glob.glob("Apriltag_images/*.png"))
print(f"[INFO] Found {len(image_files)} images")

for img_path in image_files:
    print(f"\nProcessing {img_path}")

    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print("  [WARN] Image load failed")
        continue
    
    dist_coeffs = np.array([
    -0.05380712,
     0.04227481,
     0.00049924,
     0.00054192,
    -0.01783079
    ])

    img_undist = cv2.undistort(img, K, dist_coeffs)

    detections = detector.detect(
        img_undist,
        estimate_tag_pose=True,
        camera_params=(fx, fy, cx, cy),
        tag_size=TAG_SIZE
    )

    img_color = cv2.cvtColor(img_undist, cv2.COLOR_GRAY2BGR)

    for det in detections:
        R = det.pose_R
        t = det.pose_t.reshape(3, 1)
        print(R)
        print("\t", t)
        # =========================
        # 重投影
        # =========================
        proj_points = []
        for p in tag_points_3d:
            Pc = R @ p.reshape(3, 1) + t   # tag → camera
            x = Pc[0] / Pc[2]
            y = Pc[1] / Pc[2]
            u = fx * x + cx
            v = fy * y + cy
            proj_points.append([u.item(), v.item()])

        proj_points = np.array(proj_points)
        img_points = det.corners
        # print(proj_points.shape, img_points.shape)
        # =========================
        # 计算重投影误差
        # =========================
        reproj_error = np.linalg.norm(proj_points - img_points, axis=1)
        mean_error = reproj_error.mean()

        print(f"  Tag {det.tag_id} | Mean reprojection error: {mean_error:.3f} px")

        # =========================
        # 可视化
        # =========================
        corners = img_points.astype(int)
        proj = proj_points.astype(int)

        # 真实检测角点（绿）
        for i in range(4):
            cv2.line(img_color, tuple(corners[i]),
                     tuple(corners[(i + 1) % 4]), (0, 255, 0), 2)   

        # 重投影角点（红点）
        for p in proj:
            cv2.circle(img_color, tuple(p), 4, (0, 0, 255), -1)

        # 显示误差
        cv2.putText(
            img_color,
            f"ID {det.tag_id} | err {mean_error:.2f}px",
            tuple(corners[0]),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            (255, 0, 0),
            2
        )

    # =========================
    # 显示
    # =========================
    cv2.imshow("AprilTag Reprojection Error", img_color)

    while True:
        key = cv2.waitKey(0) & 0xFF
        if key == 32:      # 空格：下一张
            break
        elif key == 27:    # ESC：退出
            cv2.destroyAllWindows()
            exit(0)

cv2.destroyAllWindows()


[INFO] Found 8 images

Processing Apriltag_images\img_000.png
[[-0.02582607  0.99961593  0.01005036]
 [-0.99962075 -0.02572745 -0.00982109]
 [-0.00955875 -0.01030019  0.99990126]]
	 [[-0.00171613]
 [ 0.02788774]
 [ 0.08963933]]
  Tag 0 | Mean reprojection error: 0.113 px

Processing Apriltag_images\img_001.png
[[ 0.44125616  0.89160983  0.10161157]
 [-0.88631058  0.41527864  0.20493218]
 [ 0.14052243 -0.180487    0.97348749]]
	 [[-0.06445742]
 [-0.00448548]
 [ 0.11575267]]
  Tag 0 | Mean reprojection error: 0.154 px

Processing Apriltag_images\img_002.png
[[ 0.39413768  0.88099306 -0.2617379 ]
 [-0.91434192  0.34708625 -0.20859047]
 [-0.09292113  0.32153129  0.94232865]]
	 [[-0.02916593]
 [-0.01989762]
 [ 0.1308325 ]]
  Tag 0 | Mean reprojection error: 0.485 px

Processing Apriltag_images\img_003.png
[[ 0.16555046  0.98498928  0.04887911]
 [-0.94634209  0.17261248 -0.2732061 ]
 [-0.27754222 -0.00102697  0.96071289]]
	 [[-0.03589875]
 [ 0.01925795]
 [ 0.10824335]]
  Tag 0 | Mean reproje